In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt
from scipy.optimize import minimize

class Boltznann_two_spins:
    """Class Boltzmann computes an optimal wave function for
    a given hamiltonian and initial w = a, b, W """
    def __init__(self, hamiltonian):
        self.hamiltonian = hamiltonian    # function which is defined outside
        self.w_optimal = np.nan    # optimal w = a, b, W
        self.psi_optimal = np.nan    # optimal psi corresponding to the optimal w
        self.min_energy = np.nan    # minimal energy corresponding to the optimal psi
        self.energy_optimization_history = []
    
    ########### PSI #################################
    
    def psi_M(self, S, w):    # Works for complex w, however, runtime errors
        """Returns a component of a wavefunction
        in the S direction"""
        a, b, W = w[0], w[1], w[2]
        psi = np.exp(np.sum(a * S))
        for i in range(len(W)):
            Fi = 2 * np.cosh(b[i] + np.sum(W[i] * S))
            psi *= Fi
        return psi
    
    def compute_psi(self, w):    # Works for complex w
        """Computes psi for w = a,b,W """
        N = len(w[0])
        psi = np.zeros((2,2), dtype = 'complex')
        psi[0, 0] = self.psi_M(np.array((0, 0)), w)
        psi[0, 1] = self.psi_M(np.array((0, 1)), w)
        psi[1, 0] = self.psi_M(np.array((1, 0)), w)
        psi[1, 1] = self.psi_M(np.array((1, 1)), w)
        return psi
    
    def compute_psi_scipy(self, w_scipy):
        """Computes psi when for a given w_scipy"""
        w = self.convert_w_from_scipy(w_scipy)
        return self.compute_psi(w)
    
    def normalize(self, psi):
        """Returns normalized psi"""
        return psi / np.sqrt(np.sum(psi ** 2))
    
    def show_psi(self, psi):    # Works for complex w
        """psi is an 2x2 array for now
        in general it will be N^N array"""
        s = 'psi = '
        for i in range(len(psi)):
            for j in range(len(psi[i])):
#                 s += str('%.5f' % psi[i,j]) + '|' + str(i) + str(j) + '>' + ' + '
                if np.imag(psi[i,j]) >= 0:
                    s += ('(' + str('%.2f' % np.real(psi[i,j])) + ' + ' + str('%.2f' % np.imag(psi[i,j])) 
                          + 'i) |' + str(i) + str(j) + '>' + ' + ')
                else:
                    s += ('(' + str('%.2f' % np.real(psi[i,j])) + ' - ' + str('%.2f' % np.imag(psi[i,j]))[1:] 
                          + 'i) |' + str(i) + str(j) + '>' + ' + ')
        print(s[:-3])
    
    def show_psi_optimal(self):
        """Shows optimal psi"""
        if self.psi_optimal is np.nan:
            raise ValueError('The optimal psi is not computed yet')
        self.show_psi(self.psi_optimal)
    
    ########### PSI #################################
    ########### OMEGA ###############################
    
    def find_len_a(self, N):
        """Returns the size of a for a given length 
        N of w = a,b,W in a form of 1D array"""
        l = 0
        while 2 * l + l ** l != N:
            l += 1
        return l
    
    def convert_w_to_scipy(self, w):
        """Converts w = a, b, W into scipy form"""
        a, b, W = w[0], w[1], w[2]
        w_scipy = np.zeros(2 * (len(a) + len(b) + len(W) * len(W[0]))) # w_scipy = [real(a[0]), imag(a[0]), real(a[1]), ...]
        for i in range(len(a)):
            w_scipy[2 * i] = np.real(a[i])
            w_scipy[2 * i + 1] = np.imag(a[i])
        for i in range(len(b)):
            w_scipy[2 * (len(a) + i)] = np.real(b[i])
            w_scipy[2 * (len(a) + i) + 1] = np.imag(b[i])
        for i in range(len(W)):
            for j in range(len(W[0])):
                w_scipy[2 * (len(a) + len(b) + i * len(W) + j)] = np.real(W[i][j])
                w_scipy[2 * (len(a) + len(b) + i * len(W) + j) + 1] = np.imag(W[i][j])
        return w_scipy
    
    def convert_w_from_scipy(self, w_scipy):
        """From w in scipy form returns w = a, b, W"""
        w = np.zeros(len(w_scipy) // 2, dtype = 'complex')
        for i in range(len(w)):
            w[i] = w_scipy[2 * i] + 1j * w_scipy[2 * i + 1]
        l = self.find_len_a(len(w))
        a, b = w[:l], w[l : 2 * l]
        W = np.resize(w[2 * l + 1:], (l, l))
        return a, b, W
    
    def show_omega_optimal(self):
        """Shows optimal w"""
        w = self.w_optimal
        a, b, W = w[0], w[1], w[2]
        print('a = ', a, '\nb = ', b, '\nW = ', W)
        
    ########### OMEGA ###############################
    ########### ENERGY ##############################
    
    def avg_energy(self, psi):
        """Returns the average energy for a given  wavefunciton
        and a given hamiltonian
        <E> = <psi|H|psi> / <psi|psi>"""
        psi_star = np.transpose(np.conjugate(psi))
#         print('PsiPsi = ', np.sum(psi_star * psi))
#         print('PsiHPsi = ', np.sum(psi_star * self.hamiltonian(psi)))
        return np.sum(psi_star * self.hamiltonian(psi)) / np.sum(psi_star * psi)
    
    def avg_energy_scipy(self, w_scipy):
        """Returns the average energy for a given configuration w,
        made to work for scipy.optimize.minimize
        Size of w is 2N, even is real, odd is complex"""
        psi = self.compute_psi_scipy(w_scipy)
        self.energy_optimization_history.append(self.avg_energy(psi))
        return self.avg_energy(psi)
    
    def show_min_energy(self):
        if self.min_energy is np.nan:
            raise ValueError('The optimal energy is not computed yet')
        s = 'E_min = ' + str(self.min_energy)
        print(s)
    
    ########### ENERGY ##############################
    ########### OPTIMIZATION ########################
    
    def find_optimal_psi(self, w0):
        """For a given hamiltonian searches for the
        ground state, i.e. the psi which minimizes the energy"""
        w0_scipy = self.convert_w_to_scipy(w0)
        w_min = minimize(self.avg_energy_scipy, w0_scipy)    # x0 is now complex, size 2N, even spots real, odd imag
        w_optimal_scipy = w_min.x
        self.w_optimal = self.convert_w_from_scipy(w_optimal_scipy)    # w_optimal is in a, b, W form
        self.psi_optimal = self.compute_psi(self.w_optimal)
        self.min_energy = self.avg_energy(self.psi_optimal)
    
    ########### OPTIMIZATION ########################
    

########### HAMILTONIAN #########################

def sigma_x(psi):
    """Applies sum sigma_x_i on a given 
    wavefunction psi and returns the 
    new wavefunciton"""
    psi1 = np.zeros((len(psi), len(psi)), dtype = 'complex')
    for i in range(len(psi)):
        for j in range(len(psi[i])):
            psi1[(i + 1) % 2, j] += psi[i, j]
            psi1[i, (j + 1) % 2] += psi[i, j]
    return psi1

def sigma_z(psi):
    """Applies sum sigma_z_i sigma_z_{i+1}
    on a given wavefunction psi and returns the 
    new wavefunciton"""
    psi1 = np.zeros((len(psi), len(psi)), dtype = 'complex')
    for i in range(len(psi)):
        for j in range(len(psi[i])):
            psi1[i, j] = (-1) ** (i + j) * psi[i, j]
    return psi1

def hamiltonian(psi):
    """Retuns wave function after the TFI Hamiltonian 
    acts on it, with h = 1 (to be modified later)"""
    return h * sigma_x(psi) + sigma_z(psi)

########### HAMILTONIAN #########################

In [ ]:
########### INITIALIZATION ########################

a = np.array((1 + 1j, 5 + 3j))
b = np.array((2 + 4j, 3 + 7j))
W = np.random.rand(2,2) + 1j * np.random.rand(2,2)
a = np.array((1, 1))
b = np.array((1, 1))
W = np.random.rand(2,2)
w = a, b, W
h = 1
boltzmann = Boltznann_two_spins(hamiltonian)

########### INITIALIZATION ########################
########### OPTIMZATION ###########################

boltzmann.find_optimal_psi(w)

########### OPTIMZATION ###########################
########### EXTRACTING PARAMETERS #################

w_optimal = boltzmann.w_optimal
psi_optimal = boltzmann.psi_optimal
min_energy = boltzmann.min_energy

########### EXTRACTING PARAMETERS #################

In [ ]:
boltzmann.show_psi_optimal()
boltzmann.show_min_energy()
boltzmann.show_omega_optimal()

In [ ]:
plt.plot(boltzmann.energy_optimization_history)
# plt.ylim(-200, 200)